# Predicting Pulsars

This project exists for multiple reasons. I am:
- endlessly fascinated by the frontier of space
- actively sharpening my data science and machine learning skills
- shamelessly trying to impress potential employers

So I dug around in the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) in hopes to find a sane, vetted, and clean dataset so I can focus on exploring. This led me to the HTRU2 data set.

## What's HTRU2?

The HTRU2 data set is a sample collected from the [High Time Resolution Universe Pulsar Survey](https://academic.oup.com/mnras/article/409/2/619/1037409)[^1] which describes possible pulsar candidates.

## Slow down. Pulsar candidates?

Pulsars are some insane celestial bodies. Take a star at least four to eight times more massive than our sun that, at the end of its life, violently explodes into a supernova. This brilliant explosion sheds its outer layers into a beautiful super nova and leaves behind an impossibly dense sphere only 12 miles across. A teaspoon of this stellar mass would weigh a billion tons. It could spin up to 43,000 revolutions per minute while emitting electromagnetic radiation. Some certain neutron stars may also happen to be highly magnetized, which would emit more of a beam. If this spinning beam crosses earth during its rotation, we'd detect a highly regular pattern of radio waves emitting from its point in our sky.

It's these patterns that the High Time Resolution Universe Pulsar Survey works to detect. Unfortunately, the radio signals picked up tend to be very noisy, and it takes a lot of time and resources to have humans annotate and verify all this data. So we're using machine learning to help aid in the process of annotation, to ease up the workload placed on the silly humans.


## What's in HTRU2?

It has attributes that describe x, y, and z. It contains 17,898 total examples, consisting of 1,639 positive examples and 16,259 negative examples. This set was verified and annotated by humans.


## So what are you doing?

I'm going to
- explore the data using descriptive statistics to get an understanding of the data
- run some initial algorithms to get a baseline performance indication
- possibly do some feature engineering to see if we can optimize the data better for the algorithms
- select the best performing algorithm
- optimize the algorithm for better performance
- then save the model

## The process

I'm using the Scientific Python tech stack.

We'll start by loading up some standard scipy libraries:

## Conclusion

This was fun!

## Footnotes

[^1]: M. J. Keith et al., 'The High Time Resolution Universe Pulsar Survey - I. System Configuration and Initial Discoveries',2010, Monthly Notices of the Royal Astronomical Society, vol. 409, pp. 619-627. DOI: 10.1111/j.1365-2966.2010.17325.x